In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import pickle

In [2]:
df = pd.read_csv('musics.csv')

In [3]:
df.dropna(inplace=True)

In [4]:
column_rename = {
    "the genre of the track": "genre",
    "Beats.Per.Minute -The tempo of the song": "bpm",
    "Energy- The energy of a song - the higher the value, the more energtic": "energy",
    "Danceability - The higher the value, the easier it is to dance to this song": "dance",
    "Loudness/dB - The higher the value, the louder the song": "loud",
    "Liveness - The higher the value, the more likely the song is a live recording": "live",
    "Valence - The higher the value, the more positive mood for the song": "valence",
    "Length - The duration of the song":"duration",
    "Acousticness - The higher the value the more acoustic the song is": "acoustic",
    "Speechiness - The higher the value the more spoken word the song contains": "speech",
    "Popularity- The higher the value the more popular the song is": "popularity"
}
df = df.rename(columns=column_rename)

In [5]:
np_df = df.select_dtypes('number').drop('year',axis=1).to_numpy()
data = StandardScaler().fit_transform(np_df)

In [6]:
similarity = cosine_similarity(data)

In [7]:
def recommend(music):
    index = df[df['title'] == music].index[0]
    distances = similarity[index]
    musics_with_index = list(enumerate(distances))
    musics_list = sorted(musics_with_index,reverse=True,key=lambda x:x[1])[1:6]
    for i in musics_list:
        music = df.iloc[i[0]]
        print(f'{music.artist} - {music.title}')
recommend('Diamonds')

One Direction - Steal My Girl
Birdy - Wings
Selena Gomez - Come & Get It
Katy Perry - Rise
Madonna - Ghosttown


In [8]:
pickle.dump(df.to_dict(),open('data.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))